# Chapter 5: Visibility and Hidden Surface Removal

## Determining What's Visible

This notebook covers:
- Z-buffer algorithm implementation
- Painter's algorithm
- Back-face culling
- View frustum culling
- Occlusion culling concepts

**Key References:** Marschner & Shirley Ch. 8-9, Gambetta Ch. 7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List

print("✓ Imports loaded")

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.z,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

print("✓ Vec3 class loaded")

In [ ]:
class ZBuffer:
    """Z-buffer (depth buffer) for hidden surface removal"""
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3), dtype=np.float32)
        self.zbuffer = np.full((height, width), float('inf'), dtype=np.float32)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        self.framebuffer[:, :] = color
        self.zbuffer[:, :] = float('inf')
    
    def set_pixel(self, x: int, y: int, depth: float, color: Tuple[float, float, float]):
        if 0 <= x < self.width and 0 <= y < self.height:
            if depth < self.zbuffer[y, x]:
                self.zbuffer[y, x] = depth
                self.framebuffer[y, x] = color
    
    def draw_triangle(self, v0: Tuple[int, int, float], 
                     v1: Tuple[int, int, float],
                     v2: Tuple[int, int, float],
                     color: Tuple[float, float, float]):
        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2
        
        min_x = max(0, min(x0, x1, x2))
        max_x = min(self.width - 1, max(x0, x1, x2))
        min_y = max(0, min(y0, y1, y2))
        max_y = min(self.height - 1, max(y0, y1, y2))
        
        def edge_function(ax, ay, bx, by, px, py):
            return (px - ax) * (by - ay) - (py - ay) * (bx - ax)
        
        area = edge_function(x0, y0, x1, y1, x2, y2)
        if abs(area) < 1e-6:
            return
        
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                w0 = edge_function(x1, y1, x2, y2, x, y)
                w1 = edge_function(x2, y2, x0, y0, x, y)
                w2 = edge_function(x0, y0, x1, y1, x, y)
                
                if w0 >= 0 and w1 >= 0 and w2 >= 0:
                    w0 /= area
                    w1 /= area
                    w2 /= area
                    z = w0 * z0 + w1 * z1 + w2 * z2
                    self.set_pixel(x, y, z, color)
    
    def display(self, title="Z-Buffer Rendering"):
        plt.figure(figsize=(10, 8))
        plt.imshow(self.framebuffer, origin='lower')
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def display_depth(self, title="Depth Buffer"):
        depth_vis = np.copy(self.zbuffer)
        finite_depths = depth_vis[np.isfinite(depth_vis)]
        if len(finite_depths) > 0:
            max_depth = np.max(finite_depths)
            depth_vis[~np.isfinite(depth_vis)] = max_depth
            min_depth = np.min(finite_depths)
            if max_depth > min_depth:
                depth_vis = (depth_vis - min_depth) / (max_depth - min_depth)
                depth_vis = 1.0 - depth_vis
        
        plt.figure(figsize=(10, 8))
        plt.imshow(depth_vis, cmap='gray', origin='lower')
        plt.title(title)
        plt.colorbar(label='Depth (normalized)')
        plt.tight_layout()
        plt.show()

class PainterRenderer:
    """Renderer using Painter's Algorithm"""
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3), dtype=np.float32)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        self.framebuffer[:, :] = color
    
    def draw_triangle(self, v0: Tuple[int, int], v1: Tuple[int, int], v2: Tuple[int, int], color: Tuple[float, float, float]):
        x0, y0 = v0
        x1, y1 = v1
        x2, y2 = v2
        
        min_x = max(0, min(x0, x1, x2))
        max_x = min(self.width - 1, max(x0, x1, x2))
        min_y = max(0, min(y0, y1, y2))
        max_y = min(self.height - 1, max(y0, y1, y2))
        
        def edge(ax, ay, bx, by, px, py):
            return (px - ax) * (by - ay) - (py - ay) * (bx - ax)
        
        area = edge(x0, y0, x1, y1, x2, y2)
        if abs(area) < 1e-6:
            return
        
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                w0 = edge(x1, y1, x2, y2, x, y)
                w1 = edge(x2, y2, x0, y0, x, y)
                w2 = edge(x0, y0, x1, y1, x, y)
                
                if w0 >= 0 and w1 >= 0 and w2 >= 0:
                    self.framebuffer[y, x] = color
    
    def render_sorted(self, triangles: List[Tuple[Tuple, Tuple, Tuple, float, Tuple]]):
        sorted_triangles = sorted(triangles, key=lambda t: t[3], reverse=True)
        for v0, v1, v2, depth, color in sorted_triangles:
            self.draw_triangle(v0, v1, v2, color)
    
    def display(self, title="Painter's Algorithm"):
        plt.figure(figsize=(10, 8))
        plt.imshow(self.framebuffer, origin='lower')
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()

def is_back_facing(v0: Vec3, v1: Vec3, v2: Vec3, view_dir: Vec3) -> bool:
    """Check if triangle is back-facing"""
    edge1 = v1 - v0
    edge2 = v2 - v0
    normal = edge1.cross(edge2)
    return normal.dot(view_dir) > 0

def is_back_facing_screen_space(x0: float, y0: float, x1: float, y1: float, x2: float, y2: float) -> bool:
    """Check if triangle is back-facing in screen space"""
    area = (x1 - x0) * (y2 - y0) - (x2 - x0) * (y1 - y0)
    return area < 0

print("✓ ZBuffer, PainterRenderer, and culling functions loaded")

---

## 1. Z-Buffer Algorithm - Theory

### 1.1 The Visibility Problem

**Problem:** When rendering 3D scenes, determine which surfaces are visible from the camera viewpoint.

**Challenges:**
- Multiple surfaces may project to the same screen pixel
- Need to determine which surface is closest (visible)
- Must handle overlapping geometry

### 1.2 Z-Buffer (Depth Buffer) Algorithm

The **Z-buffer** stores depth information for each pixel.

**Data structures:**
- **Framebuffer:** Color for each pixel $(x, y)$
- **Z-buffer:** Depth for each pixel $(x, y)$

**Algorithm:**

```
Initialize:
  for each pixel (x, y):
    framebuffer[x, y] = background_color
    zbuffer[x, y] = +∞  (or max depth)

For each triangle:
  For each pixel (x, y) in triangle's projection:
    Compute depth z at (x, y)
    if z < zbuffer[x, y]:
      zbuffer[x, y] = z
      framebuffer[x, y] = triangle_color(x, y)
```

**Depth interpolation:**

For triangle with vertices $(x_0, y_0, z_0)$, $(x_1, y_1, z_1)$, $(x_2, y_2, z_2)$ and barycentric coordinates $(\alpha, \beta, \gamma)$:

$$z(x, y) = \alpha z_0 + \beta z_1 + \gamma z_2$$

### 1.3 Z-Buffer Properties

**Advantages:**
- ✅ Simple and robust
- ✅ $O(n)$ complexity (linear in number of primitives)
- ✅ Works with any primitive ordering
- ✅ Easily implemented in hardware

**Disadvantages:**
- ❌ Requires significant memory ($W \times H$ depths)
- ❌ Z-fighting artifacts with similar depths
- ❌ No transparency support (single depth per pixel)

### 1.4 Depth Precision

**Z-buffer precision** depends on bit depth:
- 16-bit: $2^{16} = 65,536$ discrete depth values
- 24-bit: $2^{24} = 16,777,216$ depth values
- 32-bit float: Better precision

**Z-fighting:** Occurs when two surfaces have nearly identical depths, causing flickering.

**Solution:** Use larger near/far ratio, or reverse Z-buffer.

In [ ]:
# Example 1: Z-Buffer vs Painter's Algorithm Comparison
print("Example 1: Z-Buffer vs Painter's Algorithm\n")

# Create complex overlapping scene
complex_triangles = [
    # Screen coordinates (x, y, depth)
    ((50, 50, 0.8), (150, 50, 0.8), (100, 150, 0.8), (0.5, 0.2, 0.8)),
    ((120, 80, 0.5), (220, 80, 0.5), (170, 180, 0.5), (0.8, 0.5, 0.2)),
    ((200, 120, 0.7), (300, 120, 0.7), (250, 220, 0.7), (0.2, 0.8, 0.5)),
    ((100, 200, 0.4), (200, 200, 0.4), (150, 300, 0.4), (0.5, 0.5, 0.9)),
    ((250, 50, 0.6), (350, 50, 0.6), (300, 150, 0.6), (0.9, 0.5, 0.5)),
]

# Z-buffer rendering
zbuf = ZBuffer(400, 400)
zbuf.clear((0.05, 0.05, 0.1))

for (x0, y0, z0), (x1, y1, z1), (x2, y2, z2), color in complex_triangles:
    zbuf.draw_triangle(
        (int(x0), int(y0), z0),
        (int(x1), int(y1), z1),
        (int(x2), int(y2), z2),
        color
    )

print("Z-buffer: Correctly handles all overlaps")
zbuf.display("Z-Buffer: Complex Overlapping Scene")

# Painter's algorithm
painter = PainterRenderer(400, 400)
painter.clear((0.05, 0.05, 0.1))

# Convert to painter format
painter_tris = []
for (x0, y0, z0), (x1, y1, z1), (x2, y2, z2), color in complex_triangles:
    avg_depth = (z0 + z1 + z2) / 3
    painter_tris.append((
        (int(x0), int(y0)),
        (int(x1), int(y1)),
        (int(x2), int(y2)),
        avg_depth,
        color
    ))

painter.render_sorted(painter_tris)
print("Painter's: Uses depth sorting (may have issues with complex overlaps)")
painter.display("Painter's Algorithm: Same Scene")

# Example 2: Back-Face Culling Performance
print("\n\nExample 2: Back-Face Culling Performance Analysis\n")

# Create a sphere-like object (icosphere approximation)
def create_sphere_faces(subdivisions=2):
    """Create triangulated sphere"""
    # Start with icosahedron
    phi = (1 + math.sqrt(5)) / 2
    vertices = [
        Vec3(-1, phi, 0), Vec3(1, phi, 0), Vec3(-1, -phi, 0), Vec3(1, -phi, 0),
        Vec3(0, -1, phi), Vec3(0, 1, phi), Vec3(0, -1, -phi), Vec3(0, 1, -phi),
        Vec3(phi, 0, -1), Vec3(phi, 0, 1), Vec3(-phi, 0, -1), Vec3(-phi, 0, 1),
    ]
    
    # Normalize to unit sphere and scale
    vertices = [v.normalize() * 5.0 - Vec3(0, 0, 10) for v in vertices]
    
    faces = [
        (0, 11, 5), (0, 5, 1), (0, 1, 7), (0, 7, 10), (0, 10, 11),
        (1, 5, 9), (5, 11, 4), (11, 10, 2), (10, 7, 6), (7, 1, 8),
        (3, 9, 4), (3, 4, 2), (3, 2, 6), (3, 6, 8), (3, 8, 9),
        (4, 9, 5), (2, 4, 11), (6, 2, 10), (8, 6, 7), (9, 8, 1),
    ]
    
    return vertices, faces

vertices, faces = create_sphere_faces()

# Test culling
view_dir = Vec3(0, 0, -1)
culled_count = 0
visible_count = 0

for v0_idx, v1_idx, v2_idx in faces:
    v0 = vertices[v0_idx]
    v1 = vertices[v1_idx]
    v2 = vertices[v2_idx]
    
    if is_back_facing(v0, v1, v2, view_dir):
        culled_count += 1
    else:
        visible_count += 1

print(f"Sphere with {len(faces)} faces:")
print(f"  Visible (front-facing): {visible_count}")
print(f"  Culled (back-facing): {culled_count}")
print(f"  Culling efficiency: {culled_count/len(faces)*100:.1f}% reduction")

# Example 3: Depth Buffer Statistics
print("\n\nExample 3: Depth Buffer Statistics\n")

zbuf = ZBuffer(400, 400)
zbuf.clear()

# Draw multiple layers
for i in range(5):
    depth = 0.2 + i * 0.15
    color_intensity = 1.0 - i * 0.15
    
    zbuf.draw_triangle(
        (50 + i*30, 50 + i*30, depth),
        (350 - i*30, 50 + i*30, depth),
        (200, 350 - i*30, depth),
        (color_intensity, color_intensity * 0.5, color_intensity * 0.3)
    )

# Analyze depth buffer
finite_depths = zbuf.zbuffer[np.isfinite(zbuf.zbuffer)]
filled_pixels = len(finite_depths)
total_pixels = zbuf.width * zbuf.height

print(f"Depth buffer stats:")
print(f"  Resolution: {zbuf.width}x{zbuf.height} = {total_pixels:,} pixels")
print(f"  Filled pixels: {filled_pixels:,} ({filled_pixels/total_pixels*100:.1f}%)")
print(f"  Empty pixels: {total_pixels - filled_pixels:,}")
print(f"  Min depth: {np.min(finite_depths):.3f}")
print(f"  Max depth: {np.max(finite_depths):.3f}")
print(f"  Avg depth: {np.mean(finite_depths):.3f}")

zbuf.display("Layered Triangles")
zbuf.display_depth()

print("\n✓ Chapter 5 Complete!")
print("\nIn this chapter, you learned:")
print("  • Z-buffer algorithm for hidden surface removal")
print("  • Depth buffer implementation with interpolation")
print("  • Back-face culling (view-space and screen-space)")
print("  • Painter's algorithm (depth sorting)")
print("  • Comparing visibility algorithms")
print("  • Performance implications of culling")
print("\nNext Chapter: Triangle Rasterization")

---

## 4. Practical Examples

Complete demonstrations combining visibility techniques.

In [ ]:
class PainterRenderer:
    """Renderer using Painter's Algorithm (depth sorting)"""
    
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3), dtype=np.float32)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        """Clear framebuffer"""
        self.framebuffer[:, :] = color
    
    def draw_triangle(self, v0: Tuple[int, int], 
                     v1: Tuple[int, int],
                     v2: Tuple[int, int],
                     color: Tuple[float, float, float]):
        """Draw triangle without depth testing"""
        x0, y0 = v0
        x1, y1 = v1
        x2, y2 = v2
        
        # Bounding box
        min_x = max(0, min(x0, x1, x2))
        max_x = min(self.width - 1, max(x0, x1, x2))
        min_y = max(0, min(y0, y1, y2))
        max_y = min(self.height - 1, max(y0, y1, y2))
        
        # Edge function
        def edge(ax, ay, bx, by, px, py):
            return (px - ax) * (by - ay) - (py - ay) * (bx - ax)
        
        area = edge(x0, y0, x1, y1, x2, y2)
        if abs(area) < 1e-6:
            return
        
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                w0 = edge(x1, y1, x2, y2, x, y)
                w1 = edge(x2, y2, x0, y0, x, y)
                w2 = edge(x0, y0, x1, y1, x, y)
                
                if w0 >= 0 and w1 >= 0 and w2 >= 0:
                    self.framebuffer[y, x] = color
    
    def render_sorted(self, triangles: List[Tuple[Tuple, Tuple, Tuple, float, Tuple]]):
        """
        Render triangles using Painter's Algorithm
        
        Args:
            triangles: List of (v0, v1, v2, depth, color) tuples
        """
        # Sort by depth (farthest first)
        sorted_triangles = sorted(triangles, key=lambda t: t[3], reverse=True)
        
        # Draw in sorted order
        for v0, v1, v2, depth, color in sorted_triangles:
            self.draw_triangle(v0, v1, v2, color)
    
    def display(self, title="Painter's Algorithm"):
        """Display framebuffer"""
        plt.figure(figsize=(10, 8))
        plt.imshow(self.framebuffer, origin='lower')
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Test Painter's Algorithm
print("Testing Painter's Algorithm:\n")

painter = PainterRenderer(400, 400)
painter.clear((0.1, 0.1, 0.15))

# Create triangles with depth
triangles = [
    # (v0, v1, v2, depth, color)
    # Red triangle (near, should be on top)
    ((100, 100), (300, 100), (200, 300), 0.3, (1.0, 0.2, 0.2)),
    # Green triangle (middle)
    ((150, 150), (350, 150), (250, 350), 0.6, (0.2, 1.0, 0.2)),
    # Blue triangle (far, should be on bottom)
    ((50, 200), (200, 50), (350, 250), 0.9, (0.2, 0.2, 1.0)),
]

print("Triangles:")
for i, (v0, v1, v2, depth, color) in enumerate(triangles):
    color_name = ["Red", "Green", "Blue"][i]
    print(f"  {color_name}: depth={depth}")

# Render with correct sorting
print("\nRendering with Painter's Algorithm (back-to-front)...")
painter.render_sorted(triangles)
painter.display("Painter's Algorithm: Correct Sorting")

# Demonstrate problem: render in wrong order
print("\nDemonstrating incorrect order (front-to-back)...")
painter2 = PainterRenderer(400, 400)
painter2.clear((0.1, 0.1, 0.15))

# Sort front-to-back (WRONG)
wrong_order = sorted(triangles, key=lambda t: t[3])
for v0, v1, v2, depth, color in wrong_order:
    painter2.draw_triangle(v0, v1, v2, color)

painter2.display("Painter's Algorithm: Incorrect Sorting (front-to-back)")

---

## 3. Painter's Algorithm - Implementation

---

## 3. Painter's Algorithm - Theory

### 3.1 Painter's Algorithm Concept

The **Painter's Algorithm** draws polygons from back to front, like a painter painting a canvas.

**Algorithm:**
1. Sort all polygons by depth (farthest first)
2. Draw polygons in sorted order
3. Later polygons overwrite earlier ones

**Analogy:** Paint the background first, then foreground objects on top.

### 3.2 Depth Sorting

**Simple approach:** Sort by average Z-coordinate:

$$z_{\text{avg}} = \frac{z_0 + z_1 + z_2}{3}$$

Then draw from largest $z$ (farthest) to smallest $z$ (nearest).

### 3.3 Limitations

**Problems:**
- ❌ **Cyclic overlap:** Three polygons that mutually overlap (no valid ordering)
- ❌ **Intersecting polygons:** Polygons that pass through each other
- ❌ **Inefficient:** Must sort all polygons ($O(n \log n)$)

**Example cyclic overlap:**
```
Polygon A overlaps B
Polygon B overlaps C  
Polygon C overlaps A  ← No valid back-to-front order!
```

### 3.4 Advantages vs Z-Buffer

**Painter's Algorithm:**
- ✅ No z-buffer memory required
- ✅ Can support transparency (draw transparent objects last)
- ❌ Doesn't handle all cases
- ❌ Requires sorting

**Z-Buffer:**
- ✅ Handles all geometric configurations
- ✅ Simple and robust
- ❌ Requires memory
- ❌ No natural transparency support

In [ ]:
def is_back_facing(v0: Vec3, v1: Vec3, v2: Vec3, view_dir: Vec3) -> bool:
    """
    Check if triangle is back-facing using normal-view dot product
    
    Args:
        v0, v1, v2: Triangle vertices in world/view space
        view_dir: Direction from camera to triangle (or camera forward direction)
    
    Returns:
        True if back-facing (should be culled)
    """
    # Compute face normal
    edge1 = v1 - v0
    edge2 = v2 - v0
    normal = edge1.cross(edge2)
    
    # If normal points away from view direction, it's back-facing
    return normal.dot(view_dir) > 0

def is_back_facing_screen_space(x0: float, y0: float, 
                                 x1: float, y1: float,
                                 x2: float, y2: float) -> bool:
    """
    Check if triangle is back-facing in screen space using winding order
    
    Args:
        x0, y0, x1, y1, x2, y2: Screen-space triangle vertices
    
    Returns:
        True if back-facing (clockwise winding)
    """
    # Compute signed area (positive for CCW, negative for CW)
    area = (x1 - x0) * (y2 - y0) - (x2 - x0) * (y1 - y0)
    return area < 0

# Test back-face culling
print("Testing Back-Face Culling:\n")

# Create a cube with 12 triangles (2 per face)
cube_triangles = [
    # Front face (CCW when viewed from front)
    (Vec3(-1, -1, -5), Vec3(1, -1, -5), Vec3(1, 1, -5)),
    (Vec3(-1, -1, -5), Vec3(1, 1, -5), Vec3(-1, 1, -5)),
    # Back face
    (Vec3(1, -1, -7), Vec3(-1, -1, -7), Vec3(-1, 1, -7)),
    (Vec3(1, -1, -7), Vec3(-1, 1, -7), Vec3(1, 1, -7)),
    # Left face
    (Vec3(-1, -1, -7), Vec3(-1, -1, -5), Vec3(-1, 1, -5)),
    (Vec3(-1, -1, -7), Vec3(-1, 1, -5), Vec3(-1, 1, -7)),
    # Right face
    (Vec3(1, -1, -5), Vec3(1, -1, -7), Vec3(1, 1, -7)),
    (Vec3(1, -1, -5), Vec3(1, 1, -7), Vec3(1, 1, -5)),
    # Top face
    (Vec3(-1, 1, -5), Vec3(1, 1, -5), Vec3(1, 1, -7)),
    (Vec3(-1, 1, -5), Vec3(1, 1, -7), Vec3(-1, 1, -7)),
    # Bottom face
    (Vec3(-1, -1, -7), Vec3(1, -1, -7), Vec3(1, -1, -5)),
    (Vec3(-1, -1, -7), Vec3(1, -1, -5), Vec3(-1, -1, -5)),
]

# Camera looking down -Z axis
view_direction = Vec3(0, 0, -1)

print("Cube back-face culling test (camera at origin looking down -Z):\n")
front_facing_count = 0
back_facing_count = 0

for i, (v0, v1, v2) in enumerate(cube_triangles):
    is_back = is_back_facing(v0, v1, v2, view_direction)
    status = "BACK (CULLED)" if is_back else "FRONT (VISIBLE)"
    
    if is_back:
        back_facing_count += 1
    else:
        front_facing_count += 1
    
    # Calculate which face this triangle belongs to
    face_name = ["Front", "Front", "Back", "Back", "Left", "Left", 
                 "Right", "Right", "Top", "Top", "Bottom", "Bottom"][i]
    
    if i % 2 == 0:  # Only print first triangle of each face
        print(f"{face_name:6s} face: {status}")

print(f"\nSummary: {front_facing_count} visible, {back_facing_count} culled")
print(f"Culling ratio: {back_facing_count / len(cube_triangles) * 100:.1f}%")

# Test screen-space culling
print("\n\nScreen-Space Back-Face Culling Test:\n")

# CCW triangle (front-facing)
print("Triangle 1 (CCW): ", end="")
if is_back_facing_screen_space(100, 100, 200, 100, 150, 200):
    print("BACK (CULLED)")
else:
    print("FRONT (VISIBLE)")

# CW triangle (back-facing)
print("Triangle 2 (CW):  ", end="")
if is_back_facing_screen_space(100, 100, 150, 200, 200, 100):
    print("BACK (CULLED)")
else:
    print("FRONT (VISIBLE)")

---

## 2. Back-Face Culling - Implementation

---

## 2. Back-Face Culling - Theory

### 2.1 Back-Face Culling Concept

**Back-face culling** eliminates polygons facing away from the camera.

**For closed, opaque objects:** Back-facing polygons are never visible, so we can skip rendering them.

**Savings:** ~50% of polygons in typical scenes

### 2.2 Mathematical Test

Given triangle with vertices $\mathbf{v}_0, \mathbf{v}_1, \mathbf{v}_2$ and view direction $\mathbf{d}$:

**Face normal:**
$$\mathbf{n} = (\mathbf{v}_1 - \mathbf{v}_0) \times (\mathbf{v}_2 - \mathbf{v}_0)$$

**Back-face test:**
$$\mathbf{n} \cdot \mathbf{d} > 0 \implies \text{back-facing (cull)}$$

where $\mathbf{d}$ is view direction (from camera to point).

**Alternative (in screen space):**

After projection, check if triangle area is negative (counter-clockwise winding became clockwise).

### 2.3 Winding Order

**Front-facing triangles** have vertices in counter-clockwise (CCW) order when viewed from front.

**Back-facing triangles** appear clockwise (CW) after projection.

**Screen-space test:**

$$\text{Area} = \frac{1}{2}[(x_1-x_0)(y_2-y_0) - (x_2-x_0)(y_1-y_0)]$$

- $\text{Area} > 0$: CCW (front-facing)
- $\text{Area} < 0$: CW (back-facing, cull)
- $\text{Area} = 0$: Degenerate (edge-on)

In [ ]:
class ZBuffer:
    """Z-buffer (depth buffer) for hidden surface removal"""
    
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        # Framebuffer stores colors
        self.framebuffer = np.zeros((height, width, 3), dtype=np.float32)
        # Z-buffer stores depths (initialize to infinity)
        self.zbuffer = np.full((height, width), float('inf'), dtype=np.float32)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        """Clear framebuffer and z-buffer"""
        self.framebuffer[:, :] = color
        self.zbuffer[:, :] = float('inf')
    
    def set_pixel(self, x: int, y: int, depth: float, color: Tuple[float, float, float]):
        """Set pixel if depth test passes"""
        if 0 <= x < self.width and 0 <= y < self.height:
            if depth < self.zbuffer[y, x]:
                self.zbuffer[y, x] = depth
                self.framebuffer[y, x] = color
    
    def draw_triangle(self, v0: Tuple[int, int, float], 
                     v1: Tuple[int, int, float],
                     v2: Tuple[int, int, float],
                     color: Tuple[float, float, float]):
        """
        Draw triangle with depth testing
        v0, v1, v2: (x, y, z) tuples
        """
        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2
        
        # Compute bounding box
        min_x = max(0, min(x0, x1, x2))
        max_x = min(self.width - 1, max(x0, x1, x2))
        min_y = max(0, min(y0, y1, y2))
        max_y = min(self.height - 1, max(y0, y1, y2))
        
        # Edge function helper
        def edge_function(ax, ay, bx, by, px, py):
            return (px - ax) * (by - ay) - (py - ay) * (bx - ax)
        
        # Precompute area (for barycentric coordinates)
        area = edge_function(x0, y0, x1, y1, x2, y2)
        if abs(area) < 1e-6:
            return  # Degenerate triangle
        
        # Rasterize
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                # Compute barycentric coordinates
                w0 = edge_function(x1, y1, x2, y2, x, y)
                w1 = edge_function(x2, y2, x0, y0, x, y)
                w2 = edge_function(x0, y0, x1, y1, x, y)
                
                # Check if point is inside triangle
                if w0 >= 0 and w1 >= 0 and w2 >= 0:
                    # Normalize barycentric coordinates
                    w0 /= area
                    w1 /= area
                    w2 /= area
                    
                    # Interpolate depth
                    z = w0 * z0 + w1 * z1 + w2 * z2
                    
                    # Depth test and write
                    self.set_pixel(x, y, z, color)
    
    def display(self, title="Z-Buffer Rendering"):
        """Display framebuffer"""
        plt.figure(figsize=(10, 8))
        plt.imshow(self.framebuffer, origin='lower')
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def display_depth(self, title="Depth Buffer"):
        """Display depth buffer as grayscale"""
        # Normalize depth for visualization
        depth_vis = np.copy(self.zbuffer)
        # Replace inf with max finite value
        finite_depths = depth_vis[np.isfinite(depth_vis)]
        if len(finite_depths) > 0:
            max_depth = np.max(finite_depths)
            depth_vis[~np.isfinite(depth_vis)] = max_depth
            # Normalize to [0, 1]
            min_depth = np.min(finite_depths)
            if max_depth > min_depth:
                depth_vis = (depth_vis - min_depth) / (max_depth - min_depth)
                depth_vis = 1.0 - depth_vis  # Invert so near is white
        
        plt.figure(figsize=(10, 8))
        plt.imshow(depth_vis, cmap='gray', origin='lower')
        plt.title(title)
        plt.colorbar(label='Depth (normalized)')
        plt.tight_layout()
        plt.show()

# Test Z-buffer
print("Testing Z-Buffer:\n")

zbuf = ZBuffer(400, 400)
zbuf.clear((0.1, 0.1, 0.15))

# Draw overlapping triangles at different depths
# Triangle 1 (red, near)
zbuf.draw_triangle(
    (100, 100, 0.5),
    (300, 100, 0.5),
    (200, 300, 0.5),
    (1.0, 0.2, 0.2)
)

# Triangle 2 (green, middle)
zbuf.draw_triangle(
    (150, 150, 0.7),
    (350, 150, 0.7),
    (250, 350, 0.7),
    (0.2, 1.0, 0.2)
)

# Triangle 3 (blue, far)
zbuf.draw_triangle(
    (50, 200, 0.9),
    (200, 50, 0.9),
    (350, 250, 0.9),
    (0.2, 0.2, 1.0)
)

print("Drew 3 overlapping triangles at different depths")
print("Red (z=0.5), Green (z=0.7), Blue (z=0.9)")
print("Z-buffer correctly handles occlusion\n")

zbuf.display("Z-Buffer Test: Overlapping Triangles")
zbuf.display_depth("Depth Buffer Visualization")

---

## 1. Z-Buffer Algorithm - Implementation

---

## Setup and Imports

In [ ]:
# Your implementation starts here
